In [1]:
import pandas as pd

In [2]:
df_dates = pd.read_csv('./dates_daily_CDR.csv', header=None)
df_target = pd.read_csv('./results_daily_CDR.csv', header=None)

In [3]:
df_dates.head()

,0
0,1509667200
1,1509580800
2,1509494400
3,1509408000
4,1509321600


In [4]:
df_target.head()

,0,1,2,3
0,5.70,5.94,5.63,5.75
1,5.61,5.81,5.55,5.78
2,5.46,5.63,5.44,5.62
3,5.45,5.48,5.41,5.44
4,5.50,5.50,5.42,5.45


In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
normalizer = StandardScaler()

In [7]:
df = df_target
df['dates'] = df_dates

In [8]:
df.rename(columns={0: 'open', 1:'high', 2:'low',3:'close'}, inplace=True)

In [9]:
df.head()

,open,high,low,close,dates
0,5.70,5.94,5.63,5.75,1509667200
1,5.61,5.81,5.55,5.78,1509580800
2,5.46,5.63,5.44,5.62,1509494400
3,5.45,5.48,5.41,5.44,1509408000
4,5.50,5.50,5.42,5.45,1509321600


In [10]:
df.shape

(4490, 5)

In [11]:
df_train, df_test = df[490:], df[:490]

In [12]:
df_test.tail()

,open,high,low,close,dates
485,7.33,7.3501,7.145,7.15,1449014400
486,7.33,7.3950,7.300,7.36,1448928000
487,7.37,7.4000,7.260,7.32,1448841600
488,7.25,7.4000,7.240,7.36,1448582400
489,7.28,7.2800,7.210,7.26,1448409600


In [13]:
df_train.head()

,open,high,low,close,dates
490,7.22,7.260,7.13,7.26,1448323200
491,7.17,7.240,7.14,7.23,1448236800
492,7.06,7.180,7.05,7.16,1447977600
493,6.91,7.015,6.91,7.01,1447891200
494,6.79,6.920,6.76,6.91,1447804800


In [14]:
df_train.shape, df_test.shape

((4000, 5), (490, 5))

In [15]:
X_tr, y_tr = df_train['dates'], df_train['open']

In [16]:
X_te, y_te = df_test['dates'], df_test['open']

In [17]:
y_tr.values[:-1]

array([ 7.22,  7.17,  7.06, ...,  5.62,  5.62,  5.75])

In [18]:
y_tr.values[1:]

array([ 7.17,  7.06,  6.91, ...,  5.62,  5.75,  5.75])

In [19]:
diffs_tr = y_tr.values[:-1] - y_tr.values[1:]

In [20]:
grow_tr = (diffs_tr > 0).astype('int64')

In [21]:
grow_tr

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [22]:
diffs_te = y_te.values[:-1] - y_te.values[1:]

In [23]:
grow_te = (diffs_te > 0).astype('int64')

In [74]:
X_tr_grow = np.concatenate([X_tr[:-1].values.reshape(-1,1), y_tr[1:].values.reshape(-1, 1)], axis=1)
X_te_grow = np.concatenate([X_te[:-1].values.reshape(-1,1), y_te[1:].values.reshape(-1, 1)], axis=1)

In [79]:
X_tr_grow

array([[  1.44832320e+09,   7.17000000e+00],
       [  1.44823680e+09,   7.06000000e+00],
       [  1.44797760e+09,   6.91000000e+00],
       ..., 
       [  9.47116800e+08,   5.62000000e+00],
       [  9.47030400e+08,   5.75000000e+00],
       [  9.46944000e+08,   5.75000000e+00]])

In [80]:
import numpy as np

In [81]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.activations import relu, elu
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.losses import mean_absolute_error

In [82]:
X_tr_grow.shape, grow_tr.shape

((3999, 2), (3999,))

In [104]:
model = Sequential()
model.add(BatchNormalization(input_shape=(2,)))
model.add(Dense(1024, use_bias=False, activation='relu'))
model.add(Dense(1024, use_bias=False, activation='relu'))
model.add(Dense(1024, use_bias=False, activation='relu'))
model.add(Dense(256, use_bias=False, activation='relu'))
model.add(Dense(32, use_bias=False, activation='relu'))
model.add(Dense(8, use_bias=False, activation='relu'))
model.add(Dense(1, use_bias=False, activation='sigmoid'))

In [105]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_8 (Batch (None, 2)                 8         
_________________________________________________________________
dense_22 (Dense)             (None, 1024)              2048      
_________________________________________________________________
dense_23 (Dense)             (None, 1024)              1048576   
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              1048576   
_________________________________________________________________
dense_25 (Dense)             (None, 256)               262144    
_________________________________________________________________
dense_26 (Dense)             (None, 32)                8192      
_________________________________________________________________
dense_27 (Dense)             (None, 8)                 256       
__________

In [106]:
import datetime

In [107]:
model.compile('adam', 'binary_crossentropy', metrics=['acc'])

In [ ]:
model.fit(X_tr_grow, grow_tr, epochs=10000, batch_size=32)

Epoch 1/10000
3999/3999 [==============================] - 2s 431us/step - loss: 0.6465 - acc: 0.5854
Epoch 2/10000
3999/3999 [==============================] - 1s 218us/step - loss: 0.6475 - acc: 0.5951
Epoch 3/10000
3999/3999 [==============================] - 1s 220us/step - loss: 0.6443 - acc: 0.5919
Epoch 4/10000
3999/3999 [==============================] - 1s 208us/step - loss: 0.6431 - acc: 0.5861
Epoch 5/10000
3999/3999 [==============================] - 1s 224us/step - loss: 0.6453 - acc: 0.5891
Epoch 6/10000
3999/3999 [==============================] - 1s 219us/step - loss: 0.6438 - acc: 0.5851
Epoch 7/10000
3999/3999 [==============================] - 1s 209us/step - loss: 0.6404 - acc: 0.5891
Epoch 8/10000
3999/3999 [==============================] - 1s 228us/step - loss: 0.6407 - acc: 0.5934
Epoch 9/10000
3999/3999 [==============================] - 1s 220us/step - loss: 0.6425 - acc: 0.5934
Epoch 10/10000
3999/3999 [==============================] - 1s 218us/step - loss: 

3999/3999 [==============================] - 1s 214us/step - loss: 0.6385 - acc: 0.5921
Epoch 81/10000
3999/3999 [==============================] - 1s 221us/step - loss: 0.6387 - acc: 0.5931
Epoch 82/10000
3999/3999 [==============================] - 1s 221us/step - loss: 0.6388 - acc: 0.5934
Epoch 83/10000
3999/3999 [==============================] - 1s 225us/step - loss: 0.6408 - acc: 0.5931
Epoch 84/10000
3999/3999 [==============================] - 1s 228us/step - loss: 0.6380 - acc: 0.5934
Epoch 85/10000
3999/3999 [==============================] - 1s 217us/step - loss: 0.6380 - acc: 0.5951
Epoch 86/10000
3999/3999 [==============================] - 1s 209us/step - loss: 0.6387 - acc: 0.5916
Epoch 87/10000
3999/3999 [==============================] - 1s 215us/step - loss: 0.6392 - acc: 0.5934
Epoch 88/10000
3999/3999 [==============================] - 1s 215us/step - loss: 0.6392 - acc: 0.5934
Epoch 89/10000
3999/3999 [==============================] - 1s 228us/step - loss: 0.6376

3999/3999 [==============================] - 1s 219us/step - loss: 0.6395 - acc: 0.5941
Epoch 160/10000
3999/3999 [==============================] - 1s 219us/step - loss: 0.6387 - acc: 0.5956
Epoch 161/10000
3999/3999 [==============================] - 1s 238us/step - loss: 0.6384 - acc: 0.5949
Epoch 162/10000
3999/3999 [==============================] - 1s 239us/step - loss: 0.6374 - acc: 0.5946
Epoch 163/10000
3999/3999 [==============================] - 1s 222us/step - loss: 0.6388 - acc: 0.5934
Epoch 164/10000
3999/3999 [==============================] - 1s 219us/step - loss: 0.6394 - acc: 0.5911
Epoch 165/10000
3999/3999 [==============================] - 1s 231us/step - loss: 0.6385 - acc: 0.5956
Epoch 166/10000
3999/3999 [==============================] - 1s 218us/step - loss: 0.6369 - acc: 0.5956
Epoch 167/10000
3999/3999 [==============================] - 1s 219us/step - loss: 0.6381 - acc: 0.5934
Epoch 168/10000
3999/3999 [==============================] - 1s 218us/step - los

In [ ]:
# row = '2017-11-07'
# timestamp_as_list = row.split('-')
# year = int(timestamp_as_list[0])
# month = int(timestamp_as_list[1])
# day = int(timestamp_as_list[2])
# date = datetime.datetime(year=year, month=month, day=day, tzinfo=datetime.timezone.utc)  # Convert to UTC time
# unix_time = int(date.timestamp())

In [ ]:
# model.predict(np.array([[unix_time, 5.8600]]))

In [88]:
prediction = model.predict_classes(X_te_grow)

489/489 [==============================] - 0s 327us/step


In [89]:
prediction_tr = model.predict_classes(X_tr_grow)

3999/3999 [==============================] - 0s 47us/step


In [90]:
prediction_tr

array([[0],
       [0],
       [0],
       ..., 
       [0],
       [0],
       [0]])

In [91]:
from sklearn.metrics import classification_report, confusion_matrix

In [92]:
print(classification_report(grow_tr, prediction_tr))
print(confusion_matrix(grow_tr, prediction_tr))

             precision    recall  f1-score   support

          0       0.60      0.98      0.74      2373
          1       0.54      0.03      0.06      1626

avg / total       0.57      0.60      0.47      3999

[[2327   46]
 [1572   54]]


In [93]:
print(classification_report(grow_te, prediction))
print(confusion_matrix(grow_te, prediction))

             precision    recall  f1-score   support

          0       0.53      0.67      0.59       258
          1       0.49      0.35      0.41       231

avg / total       0.51      0.52      0.51       489

[[172  86]
 [150  81]]
